In [ ]:
from ragas import evaluate
from datasets import Dataset

import os
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores.faiss import FAISS
from langchain.chains.retrieval_qa.base import RetrievalQA
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    context_entity_recall,
    answer_similarity,
    answer_correctness
    )

In [ ]:
from RAG import (
    extract_docs,
    generate_prompt_llama,
    generate_prompt_mixtral,
    get_embeddings,
    get_llm,
    get_vector_store
    )

In [ ]:
DEVICE = "mps:0"

In [ ]:
embeddings_model = "Alibaba-NLP/gte-large-en-v1.5"
embeddings_chunk_size = 1024
embeddings = get_embeddings(embeddings_model)

In [ ]:
environment_sensing_dataset = '../dataset/environment-sensing-dataset'
txt_glob = "**/*txt"
vector_store_file_name = '../faiss-db/faiss_index'

In [ ]:
if os.path.exists(vector_store_file_name):
    print("Vector store exists")
    vector_store = FAISS.load_local(vector_store_file_name, embeddings, allow_dangerous_deserialization=True)
    print("Vector store loaded")
else:
    print("Vector store does not exist, creating vector store...")
    print("Extracting documents...")
    print("\tExtracting txts...")

    extracted_docs = extract_docs(environment_sensing_dataset, txt_glob, embeddings_chunk_size)

    vector_store = get_vector_store(extracted_docs, embeddings)
    vector_store.save_local(vector_store_file_name)
    print("Vector store saved and loaded")

In [ ]:
extracted_docs = extract_docs(environment_sensing_dataset, txt_glob, embeddings_chunk_size)

In [ ]:
def get_conversation_chain_llama(llm, vector_store):
    SYSTEM_PROMPT = (
        "Use the following pieces of context, along with the chat history if there is any, to answer the question at "
        "the end. If"
        "you don't know the answer, just say that you don't know, don't try to make up an answer.")

    template_llama = generate_prompt_llama("""
            {context}
            

            Question: {question}
            """.strip(), system_prompt=SYSTEM_PROMPT)

    prompt = PromptTemplate(template=template_llama, input_variables=["context", "question"])
    return RetrievalQA.from_llm(
            llm=llm,
            retriever=vector_store.as_retriever(),
            prompt=prompt,
            return_source_documents=True
            )

In [ ]:
def get_conversation_chain_mistral(llm, vector_store):
    SYSTEM_PROMPT = (
        "Use the following pieces of context, along with the chat history if there is any, to answer the question at "
        "the end. If"
        "you don't know the answer, just say that you don't know, don't try to make up an answer.")

    template_mixtral = generate_prompt_mixtral("""
            {context}
            

            Question: {question}
            """.strip(), system_prompt=SYSTEM_PROMPT)

    prompt = PromptTemplate(template=template_mixtral, input_variables=["context", "question"])
    return RetrievalQA.from_llm(
            llm=llm,
            retriever=vector_store.as_retriever(),
            prompt=prompt,
            return_source_documents=True
            )

In [ ]:
llm_mistral_7b = get_llm("mistralai/Mixtral-8x7B-Instruct-v0.1")
conversational_chain_mistral_7b = get_conversation_chain_mistral(llm_mistral_7b, vector_store)

llm_mixtral_8x7b = get_llm("mistralai/Mixtral-8x7B-Instruct-v0.1")
conversational_chain_mixtral_8x7b = get_conversation_chain_mistral(llm_mixtral_8x7b, vector_store)

llm_llama_8b = get_llm("meta-llama/Meta-Llama-3-8B-Instruct")
conversational_chain_llama_8b = get_conversation_chain_llama(llm_llama_8b, vector_store)

llm_llama_70b = get_llm("meta-llama/Meta-Llama-3-70B-Instruct")
conversational_chain_llama_70b = get_conversation_chain_llama(llm_llama_70b, vector_store)

llm_llama_405b = get_llm("meta-llama/Meta-Llama-3.1-405B-Instruct-FP8")
conversational_chain_llama_405b = get_conversation_chain_llama(llm_llama_405b, vector_store)

In [ ]:
questions = [
    "Given the following information captured from unit1, describe the physical environment this information provides and be as detailed as possible. Do your best to estimate distances of objects and their types, if there are any, and if there are any blockages from obstacles, cars, pedestrians or anything that can cause obstacles. Unit1 is a vehicle with a 360 camera (two 180 degree cameras), 360 degree 3D lidar, and a GPS module, unit2 is a vehicle mounted with a GPS module and a transmitter. The receiver (Unit 1) is 34.453423627567986 meters away at a bearing of 3.610133544479254 degrees from the transmitter (Unit 2). The transmitter (Unit 2) is 34.453423627567986 meters away at a bearing of 3.610133544479254 degrees from the receiver (Unit 1). LiDAR object detection results: A Vehicle is detected 19.44 meters away at a direction of 295.38 degrees. Scene details from camera: The scene depicts a view from the front seat of a car driving down a city street. The car is parked on the side of the road, and the driver appears to be standing next to it. There are several buildings visible in the background, including a tall building on the right side of the image. Additionally, there is a pedestrian walking on the sidewalk near the car. The sun is shining brightly in the sky, casting a warm and inviting glow over the scene. The scene depicts a view from the back seat of a car driving down a city street. There are several buildings visible in the background, including a tall building on the left side of the image and a shorter building on the right side. In the foreground, there is a bottle of water placed on the dashboard of the car. Additionally, there are two people standing on the sidewalk, one on the left side and one on the right side of the street.",

    "Given the following information captured from unit1, describe the physical environment this information provides and be as detailed as possible. Do your best to estimate distances of objects and their types, if there are any, and if there are any blockages from obstacles, cars, pedestrians or anything that can cause obstacles. Unit1 is a vehicle with a 360 camera (two 180 degree cameras), 360 degree 3D lidar, and a GPS module, unit2 is a vehicle mounted with a GPS module and a transmitter. The transmitter (Unit 2) is 8.066753628120216 meters away at a bearing of 182.27478933305133 degrees from the receiver (Unit 1). LiDAR object detection results: A Vehicle is detected 15.07 meters away at a direction of 322.89 degrees. A Vehicle is detected 11.38 meters away at a direction of 229.71 degrees. A Vehicle is detected 9.83 meters away at a direction of 241.23 degrees. A Vehicle is detected 16.96 meters away at a direction of 181.47 degrees. Scene details from camera: The scene depicts a man wearing a red shirt standing in the middle of a city street near a parking lot. There are several cars parked in the parking lot, and some of them are visible in the background. The man appears to be gesturing with his hand, possibly indicating something to the driver of the car he is standing next to. In addition to the man and the parked cars, there are several trees scattered throughout the scene. One tree can be seen on the left side of the image, while another is located on the right side. There are also two benches present, one on the left side and another on the right side of the image. The scene is a bird's-eye view of a parking lot from the perspective of a car. There are several vehicles parked in the lot, including a blue car, a white car, and a truck. Additionally, there are some trees visible in the background, providing a scenic setting for the parking lot. The sun is shining brightly, casting a warm and inviting glow over the area.",

    "Given the following information captured from unit1, describe the physical environment this information provides and be as detailed as possible. Do your best to estimate distances of objects and their types, if there are any, and if there are any blockages from obstacles, cars, pedestrians or anything that can cause obstacles. Unit1 is a vehicle with a 360 camera (two 180 degree cameras), 360 degree 3D lidar, and a GPS module, unit2 is a vehicle mounted with a GPS module and a transmitter. The transmitter (Unit 2) is 112.79471527640717 meters away from the receiver (Unit 1) and Unit 1 is at a bearing of 359.8653702653738 degrees from Unit 2's position. LiDAR object detection results: A Vehicle is detected 3.21 meters away at a direction of 24.82 degrees. A Vehicle is detected 15.41 meters away at a direction of 158.79 degrees. Scene details from camera: The scene depicts a highway with two vehicles, an ambulance and a car, traveling in opposite directions. The ambulance is driving on the left side of the road, while the car is driving on the right side of the road. The ambulance has a siren blaring, indicating that it is responding to an emergency call. In the background, there is a bridge spanning over the highway. The scene depicts a highway with an ambulance parked on the side of the road. The ambulance is visible from the back of the car, which is traveling in the same direction as the ambulance. In addition to the ambulance, there are several other vehicles on the road, including cars, trucks, and vans.",

    "Given the following information captured from unit1, describe the physical environment this information provides and be as detailed as possible. Do your best to estimate distances of objects and their types, if there are any, and if there are any blockages from obstacles, cars, pedestrians or anything that can cause obstacles. Unit1 is a vehicle with a 360 camera (two 180 degree cameras), 360 degree 3D lidar, and a GPS module, unit2 is a vehicle mounted with a GPS module and a transmitter. The transmitter (Unit 2) is 8.08363553211514 meters away at a bearing of 182.445600684427 degrees from the receiver (Unit 1). LiDAR object detection results: A Vehicle is detected 11.37 meters away at a direction of 229.76 degrees. A Vehicle is detected 16.97 meters away at a direction of 181.48 degrees. A Vehicle is detected 9.83 meters away at a direction of 241.20 degrees. Scene details from camera: The scene depicts a person standing in the middle of a parking lot, with a car parked nearby. The person is wearing a red shirt and appears to be waving or gesturing towards the car. There are several trees scattered throughout the parking lot, adding a natural touch to the environment. In the background, there is a large building, possibly a hotel or an apartment complex, which can be seen from the vantage point of the car's front window. The scene is a bird's-eye view of a parking lot from the perspective of a car. There are several vehicles parked in the lot, including a blue car, a red car, and a white car. Additionally, there are some trees visible in the background, providing a scenic setting for the parking lot. The sun is shining brightly in the sky, casting a warm and inviting glow over the entire scene.",

    "Given the following information captured from unit1, describe the physical environment this information provides and be as detailed as possible. Do your best to estimate distances of objects and their types, if there are any, and if there are any blockages from obstacles, cars, pedestrians or anything that can cause obstacles. Unit1 is a vehicle with a 360 camera (two 180 degree cameras), 360 degree 3D lidar, and a GPS module, unit2 is a vehicle mounted with a GPS module and a transmitter. The transmitter (Unit 2) is 8.204789594875955 meters away at a bearing of 182.09057338850562 degrees from the receiver (Unit 1). LiDAR object detection results: A Vehicle is detected 11.42 meters away at a direction of 229.72 degrees. A Vehicle is detected 9.71 meters away at a direction of 242.43 degrees. A Vehicle is detected 17.00 meters away at a direction of 181.41 degrees. Scene details from camera: The scene is a bird's-eye view of a parking lot from the perspective of a car driving through it. There are several cars parked in the lot, some of which are closer to the camera, while others are farther away. A building can be seen in the background, providing context for the parking lot's location. In addition to the cars and building, there are several trees scattered throughout the parking lot, contributing to the overall visual appeal of the scene. The scene is a bird's-eye view of a parking lot from the perspective of a car. There are several vehicles parked in the lot, including a blue car, a red car, and a white car. Additionally, there are some trees and buildings visible in the background. The sun is shining brightly in the sky, casting a warm and inviting glow over the scene.",

    "Given the following information captured from unit1, describe the physical environment this information provides and be as detailed as possible. Do your best to estimate distances of objects and their types, if there are any, and if there are any blockages from obstacles, cars, pedestrians or anything that can cause obstacles. Unit1 is a vehicle with a 360 camera (two 180 degree cameras), 360 degree 3D lidar, and a GPS module, unit2 is a vehicle mounted with a GPS module and a transmitter. The receiver (Unit 1) is 9.518030283574547 meters away at a bearing of 358.69660658751474 degrees from the transmitter (Unit 2). LiDAR object detection results: A Vehicle is detected 14.37 meters away at a direction of 338.02 degrees. A Vehicle is detected 16.26 meters away at a direction of 181.69 degrees. Scene details from camera: The scene depicts a city street with two cars parked on the side of the road. One of the cars is white, while the other is black. There is also a bus driving down the street, passing by the parked cars. In addition to the cars and bus, there are several people visible in the scene. Two people can be seen walking on the sidewalk on the left side of the street, while another person is standing on the sidewalk on the right side of the street. Overall, the image captures a bustling urban environment with vehicles and pedestrians moving along the street. The scene depicts a view from the back of a car as it drives down a busy city street. There are multiple vehicles on the road, including cars, trucks, and motorcycles, creating a congested traffic flow. In addition to the vehicles, there are several pedestrians walking along the sidewalks, adding to the bustling atmosphere. Some of the pedestrians are closer to the camera, while others are further away. A building can also be seen in the background, adding to the urban setting.",

    "Given the following information captured from unit1, describe the physical environment this information provides and be as detailed as possible. Do your best to estimate distances of objects and their types, if there are any, and if there are any blockages from obstacles, cars, pedestrians or anything that can cause obstacles. Unit1 is a vehicle with a 360 camera (two 180 degree cameras), 360 degree 3D lidar, and a GPS module, unit2 is a vehicle mounted with a GPS module and a transmitter. The transmitter (Unit 2) is 134.9016963054195 meters away from the receiver (Unit 1) where Unit 1 is at a bearing of 178.86933709163418 degrees from Unit 2. LiDAR object detection results: A Vehicle is detected 15.51 meters away at a direction of 66.55 degrees. A Vehicle is detected 22.92 meters away at a direction of 29.51 degrees. A Vehicle is detected 18.97 meters away at a direction of 80.52 degrees. A Vehicle is detected 12.68 meters away at a direction of 201.62 degrees. A Vehicle is detected 3.70 meters away at a direction of 205.08 degrees. A Vehicle is detected 26.97 meters away at a direction of 116.24 degrees. Scene details from camera: The scene depicts a car driving down a city street, as seen from the driver's point of view. The car is parked on the side of the road, and there are several other vehicles visible in the background. The sun is shining brightly in the sky, casting a warm and inviting glow over the scene. Additionally, there are trees lining the street, adding a natural touch to the urban environment. The scene depicts a view from the back of a car as it drives down a busy city street. There are multiple cars visible in the scene, with some parked on the side of the road and others passing by. In addition to the cars, there are several pedestrians walking along the sidewalks, adding to the bustling atmosphere of the city. Some of the pedestrians are closer to the camera, while others are further away. A traffic light can be seen in the distance, indicating the flow of traffic on the street.",

    "Given the following information captured from unit1, describe the physical environment this information provides and be as detailed as possible. Do your best to estimate distances of objects and their types, if there are any, and if there are any blockages from obstacles, cars, pedestrians or anything that can cause obstacles. Unit1 is a vehicle with a 360 camera (two 180 degree cameras), 360 degree 3D lidar, and a GPS module, unit2 is a vehicle mounted with a GPS module and a transmitter. The receiver (Unit 1) is 16.452267066094496 meters away at a bearing of 171.16386919015633 degrees from the transmitter (Unit 2). LiDAR object detection results: A Vehicle is detected 10.09 meters away at a direction of 345.35 degrees. A Vehicle is detected 18.56 meters away at a direction of 319.09 degrees. A Vehicle is detected 10.38 meters away at a direction of 204.80 degrees. Scene details from camera: The scene is captured from the front view of a car driving down a city street. There are several vehicles on the road, including cars, trucks, and buses, creating a bustling urban environment. The sun is shining brightly in the sky, casting a warm and inviting glow over the scene. In addition to the vehicles, there are several buildings visible in the background, contributing to the vibrant atmosphere of the city. The scene depicts a view from the back of a car as it drives down a city street. There are several vehicles on the road, including a white car, a blue car, and a black truck. Additionally, there are two motorcycles in the scene, one behind the white car and the other behind the black truck. A traffic light can be seen in the distance, indicating that the cars are approaching an intersection.",

    "Given the following information captured from unit1, describe the physical environment this information provides and be as detailed as possible. Do your best to estimate distances of objects and their types, if there are any, and if there are any blockages from obstacles, cars, pedestrians or anything that can cause obstacles. Unit1 is a vehicle with a 360 camera (two 180 degree cameras), 360 degree 3D lidar, and a GPS module, unit2 is a vehicle mounted with a GPS module and a transmitter. The transmitter (Unit 2) is 30.478007928925432 meters away at a bearing of 5.203883054678897 degrees from the receiver (Unit 1). LiDAR object detection results: A Vehicle is detected 15.08 meters away at a direction of 20.64 degrees. A Vehicle is detected 9.64 meters away at a direction of 187.22 degrees. A Vehicle is detected 7.73 meters away at a direction of 158.20 degrees. Scene details from camera: The scene is captured from the front view of a car traveling down a busy highway. There are multiple vehicles on the road, including cars, trucks, and buses, creating a congested traffic flow. The highway is lined with palm trees, adding a scenic touch to the urban environment. In the background, there is a blue sky visible, providing a clear and sunny day for the driver's journey. Overall, the image captures the hustle and bustle of a busy highway, showcasing the diversity of vehicles and the presence of palm trees along the roadside. The scene depicts a car driving down a freeway with a blue IKEA building visible in the background. The car is traveling at a moderate speed, and there are several other vehicles on the road as well. Some of these vehicles are closer to the camera, while others are farther away. In addition to the cars, there are two trucks visible in the scene. One truck is parked on the side of the road, while the other is driving on the freeway. There are also several trees lining the sides of the road, adding to the scenic view.",

    "Given the following information captured from unit1, describe the physical environment this information provides and be as detailed as possible. Do your best to estimate distances of objects and their types, if there are any, and if there are any blockages from obstacles, cars, pedestrians or anything that can cause obstacles. Unit1 is a vehicle with a 360 camera (two 180 degree cameras), 360 degree 3D lidar, and a GPS module, unit2 is a vehicle mounted with a GPS module and a transmitter. The transmitter (Unit 2) is 30.78343479907267 meters away at a bearing of 3.655295658299565 degrees from the receiver (Unit 1). LiDAR object detection results: A Vehicle is detected 8.18 meters away at a direction of 11.21 degrees. A Vehicle is detected 9.27 meters away at a direction of 189.68 degrees. A Vehicle is detected 8.45 meters away at a direction of 163.50 degrees. Scene details from camera: The scene depicts a busy highway with multiple cars driving on it. There are at least three cars visible in the image, including a white car, a blue car, and a black car. Additionally, there is a truck parked on the side of the road, which adds to the diversity of vehicles on the highway. Traffic lights can be seen in the distance, indicating the presence of intersections along the road. The view from the cars front windshield provides a birds-eye perspective of the busy highway and its surroundings. The scene is captured from a back POV of a car traveling down a busy highway. There are multiple vehicles on the road, including cars, trucks, and buses, creating a congested traffic flow. The sun is shining brightly in the sky, casting a warm and inviting glow over the scene. In addition to the vehicles, there are several buildings visible in the background, providing context for the location.",

    "Given the following information captured from unit1, describe the physical environment this information provides and be as detailed as possible. Do your best to estimate distances of objects and their types, if there are any, and if there are any blockages from obstacles, cars, pedestrians or anything that can cause obstacles. Unit1 is a vehicle with a 360 camera (two 180 degree cameras), 360 degree 3D lidar, and a GPS module, unit2 is a vehicle mounted with a GPS module and a transmitter. The transmitter (Unit 2) is 10.539957539355683 meters away at a bearing of 159.9010115892885 degrees from the receiver (Unit 1). LiDAR object detection results: A Vehicle is detected 21.48 meters away at a direction of 314.58 degrees. A Vehicle is detected 24.33 meters away at a direction of 336.26 degrees. A Vehicle is detected 11.60 meters away at a direction of 168.20 degrees. A Vehicle is detected 15.38 meters away at a direction of 188.91 degrees. A Vehicle is detected 23.36 meters away at a direction of 163.50 degrees. Scene details from camera: The scene depicts a car driving underneath a bridge on a sunny day. There are several cars parked along the side of the road, and a traffic light can be seen in the distance. The camera is positioned from the front of the car, providing a bird's-eye view of the surrounding area. In the scene, there are several cars driving on a busy city street underneath an overpass. The view is captured from the back of a car, providing a bird's-eye view of the traffic below. There are multiple cars visible in the image, some of which are parked on the side of the road, while others are moving along the street. Traffic lights can be seen in the distance, indicating the flow of traffic and the need for drivers to follow the rules of the road. Overall, the scene captures the hustle and bustle of a busy city street, showcasing the diverse range of vehicles and pedestrians navigating their way through the urban environment.",

    "LiDAR object detection results: A Vehicle is detected 15.08 meters away at a direction of 20.64 degrees. A Vehicle is detected 9.64 meters away at a direction of 187.22 degrees. A Vehicle is detected 7.73 meters away at a direction of 158.20 degrees. Based on the LiDAR object detection results from Unit 1, what is the distance and direction of the vehicle detected behind Unit 1 while it was capturing data on the highway?",

    "LiDAR object detection results: A Vehicle is detected 15.08 meters away at a direction of 20.64 degrees. A Vehicle is detected 9.64 meters away at a direction of 187.22 degrees. A Vehicle is detected 7.73 meters away at a direction of 158.20 degrees. Scene details from camera: The scene is captured from the front view of a car traveling down a busy highway. There are multiple vehicles on the road, from the front view of a car traveling down a busy highway. There are multiple vehicles on the road, including cars, trucks, and buses, creating a congested traffic flow. The highway is lined with palm trees, adding a scenic touch to the urban environment. In the background, there is a blue sky visible, providing a clear and sunny day for the driver's journey. Overall, the image captures the hustle and bustle of a busy highway, showcasing the diversity of vehicles and the presence of palm trees along the roadside.The scene depicts a car driving down a freeway with a blue IKEA building visible in the background. The car is traveling at a moderate speed, and there are several other vehicles on the road as well. Some of these vehicles are closer to the camera, while others are farther away. In addition to the cars, there are two trucks visible in the scene. One truck is parked on the side of the road, while the other is driving on the freeway. There are also several trees lining the sides of the road, adding to the scenic view. According to the scene details captured from the front view of Unit 1 while traveling down a busy highway, what is the estimated distance between Unit 1 and the red car in the center lane?",

    "Scene details from camera: The scene depicts a person standing in the middle of a parking lot, with a car parked nearby. The person is wearing a red shirt and in the middle of a parking lot, with a car parked nearby. The person is wearing a red shirt and appears to be waving or gesturing towards the car. There are several trees scattered throughout the parking lot, adding a natural touch to the environment. In the background, there is a large building, possibly a hotel or an apartment complex, which can be seen from the vantage point of the car's front window.The scene is a bird's-eye view of a parking lot from the perspective of a car. There are several vehicles parked in the lot, including a blue car, a red car, and a white car. Additionally, there are some trees visible in the background, providing a scenic setting for the parking lot. The sun is shining brightly in the sky, casting a warm and inviting glow over the entire scene. While capturing data in an urban parking lot, are there any trees nearby Unit 1, based on the scene details provided?",

    "LiDAR object detection results: A Vehicle is detected 14.37 meters away at a direction of 338.02 degrees. A Vehicle is detected 16.26 meters away at a direction of 181.69 degrees. Scene details from camera: The scene depicts a city street with two cars parked on the side of the road. One of the cars is white, while the other is black. There is also a bus driving down the street, passing by the cars is white, while the other is black. There is also a bus driving down the street, passing by the parked cars. In addition to the cars and bus, there are several people visible in the scene. Two people can be seen walking on the sidewalk on the left side of the street, while another person is standing on the sidewalk on the right side of the street. Overall, the image captures a bustling urban environment with vehicles and pedestrians moving along the street.The scene depicts a view from the back of a car as it drives down a busy city street. There are multiple vehicles on the road, including cars, trucks, and motorcycles, creating a congested traffic flow. In addition to the vehicles, there are several pedestrians walking along the sidewalks, adding to the bustling atmosphere. Some of the pedestrians are closer to the camera, while others are further away. A building can also be seen in the background, adding to the urban setting. In the last captured scene showing a bustling urban street, what is the estimated distance of the bus from Unit 1 as it passed by the parked cars?"
    ]

In [ ]:
ground_truths = [
    """
    Physical Environment Description
    
    Scene Overview
    
    Location: The environment is an urban city street characterized by multiple buildings, vehicles, and pedestrians.
    
    Weather: Clear skies with bright sunlight, indicating a sunny day.
    
    Objects and Distances
    
    1. Vehicles:
       - White Car: Positioned in the middle of the street in the first image. Estimated distance: approximately 8-10 meters from Unit 1.
       - Blue Car: Visible farther down the street in the second image. Estimated distance: approximately 30-35 meters from Unit 1.
    
    2. Pedestrians:
       - First Image: One person standing on the sidewalk near the car. Estimated distance: approximately 5-7 meters from Unit 1.
       - Second Image: Three people waiting at a bus stop on the left side of the street. Estimated distance: approximately 15-20 meters from Unit 1.
    
    3. Buildings:
       - Left Side: Several buildings.
       - Right Side: Multiple buildings, including a tall building and several lower structures.
    
    4. Traffic Lights and Signs:
       - Traffic lights are visible at the intersection ahead, approximately 50-60 meters away.
       - Street signs provide additional context for the urban environment.
    
    LiDAR Object Detection Results
    
    Vehicle Detection:
       - 19.44 meters away at 295.38 degrees: Likely a vehicle positioned in the upper left quadrant relative to Unit 1.
    
    GPS Module Information
    
    Transmitter (Unit 2):
       - Distance: 34.45 meters away.
       - Bearing: 3.61 degrees from Unit 1 (essentially directly north).
    
    Potential Obstacles
    
    1. Vehicles:
       - Multiple vehicles at varying distances can create potential obstacles, causing blockage or reflection of signals.
    
    2. Pedestrians:
       - Pedestrians can cause temporary signal blockage or reflection, adding dynamic elements to the environment.
    
    3. Buildings and Structures:
       - Buildings on either side of the street can obstruct the line of sight (LoS) and attenuate signals, especially those in higher frequency ranges.
    
    Environmental Factors
    
    Clear Weather: Beneficial for wireless communication with minimal signal attenuation due to atmospheric conditions.
    
    Sunlight: Bright sunlight can cause glare, affecting optical sensors and cameras.
    
    Summary of Scene
    
    The physical environment includes a well-lit urban city street with multiple buildings, vehicles, and pedestrians. The parked and moving vehicles, as well as the pedestrians, serve as potential obstacles, impacting the line of sight for wireless communication between Unit 1 and Unit 2. The buildings on either side of the street can introduce additional interference and attenuation effects. Overall, the clear weather aids in maintaining good signal propagation conditions, though the dynamic nature of the environment with vehicles and pedestrians can cause fluctuations in signal quality.
    """,

    """
    Physical Environment Description
    
    Scene Overview
    
    Location: An outdoor urban area, likely part of a campus or business district, characterized by a parking lot with adjacent buildings and greenery.
    
    Weather: Clear skies with bright sunlight, indicating a sunny day.
    
    Objects and Distances
    
    1. Man in Red Shirt:
       - Position: Standing in the middle of the street near a parking lot.
       - Distance to Unit 1: Approximately 5 meters, based on the image perspective.
       - Action: Gesturing with his hand, possibly communicating with the driver of a nearby vehicle.
    
    2. Vehicles:
       - Blue Car: Directly in front of Unit 1, parked. Estimated distance: approximately 8 meters.
       - White Car: Visible in the parking lot, partially obscured by trees. Estimated distance: around 10-12 meters.
       - Truck: Positioned further back, approximately 15-17 meters from Unit 1.
    
    3. Trees and Greenery:
       - Left Side: One tree is directly next to the parking lot.
       - Right Side: Several trees are providing shade and contributing to the scenic setting.
    
    4. Benches:
       - One bench on the left side and another on the right side, indicating a pedestrian-friendly area.
    
    LiDAR Object Detection Results
    
    1. Vehicle Detection:
       - 15.07 meters away at 322.89 degrees: Likely in the upper right quadrant relative to Unit 1.
       - 11.38 meters away at 229.71 degrees: Positioned in the lower left quadrant.
       - 9.83 meters away at 241.23 degrees: Also in the lower left quadrant, closer than the previous vehicle.
       - 16.96 meters away at 181.47 degrees: Directly behind Unit 1.
    
    GPS Module Information
    
    1. Transmitter (Unit 2):
       - Distance: 8.07 meters away.
       - Bearing: 182.27 degrees from Unit 1 (essentially directly south).
    
    Potential Obstacles
    
    1. Vehicles:
       - Multiple parked vehicles at varying distances can create potential obstacles, causing blockage or reflection of signals.
    
    2. Trees and Greenery:
       - Trees can obstruct the line of sight (LoS) and attenuate signals, especially those in higher frequency ranges (e.g., 2.4 GHz and above).
    
    3. Pedestrian (Man in Red Shirt):
       - As a temporary obstacle, he may cause minor fluctuations in signal quality due to blockage or reflection.
    
    Environmental Factors
    
    1. Clear Weather: Beneficial for wireless communication with minimal signal attenuation due to atmospheric conditions.
    
    2. Sunlight: While it does not directly impact RF signals, bright sunlight can cause glare affecting optical sensors and cameras.
    
    Summary of Scene
    
    The physical environment includes a well-lit urban parking lot with multiple parked vehicles, scattered trees, and some pedestrian amenities like benches. The vehicles and trees serve as potential obstacles, impacting the line of sight for wireless communication between Unit 1 and Unit 2. The man in the red shirt adds a temporary dynamic element that could briefly obstruct signals. Overall, the clear weather aids in maintaining good signal propagation conditions, though the vehicles and trees can introduce some interference and attenuation effects.
    """,

    """
Physical Environment Description with Focus on Nearby Vehicles as Obstacles

Scene Overview
- Location: The environment is a highway, characterized by multiple lanes, vehicles, and a desert landscape with sparse vegetation.
- Weather: Clear skies with bright sunlight, indicating a sunny day.

Objects and Distances
1. Vehicles:
   - Ambulance:
     - First Image: Visible in the right lane, very close to Unit 1, approximately 3-5 meters away. The proximity of the ambulance poses a significant obstacle, potentially blocking and reflecting signals.
     - Second Image: Another view of the ambulance (or the same one), still close, about 5-7 meters away. This continued closeness further emphasizes its role as an obstacle.
   - Cars:
     - First Image: Blue car visible ahead, approximately 30-40 meters away.
     - Second Image: Multiple vehicles, including a white truck, approximately 50-60 meters ahead.

2. Bridges and Overpasses:
   - Second Image: An overpass is visible, spanning across the highway. Estimated distance: approximately 100-150 meters ahead.

3. Vegetation and Landscape:
   - Sparse desert vegetation with small bushes and trees lining the highway.
   - First Image: Vegetation visible on the left side of the highway, approximately 20-30 meters from the road.

LiDAR Object Detection Results
- Vehicle Detection:
  - 3.21 meters away at 24.82 degrees: Likely the ambulance in the right lane, very close to Unit 1, creating a strong signal obstacle.
  - 15.41 meters away at 158.79 degrees: Another vehicle detected behind Unit 1, possibly in the left lane.

GPS Module Information
- Transmitter (Unit 2):
  - Distance: 112.79 meters away.
  - Bearing: 359.87 degrees from Unit 1 (essentially directly north).

Impact of Nearby Vehicles as Obstacles
1. Signal Blockage:
   - Ambulance Proximity: The ambulance being as close as 3-5 meters significantly blocks the line of sight (LoS) for wireless signals. This obstruction can result in signal shadowing, where the signal cannot pass directly through the large metal body of the vehicle, leading to areas of weak or no signal immediately behind it.
   
2. Signal Reflection and Multipath Interference:
   - Metal Surfaces: The metal surfaces of the nearby ambulance and other vehicles can reflect signals, causing multipath interference. This occurs when the signal bounces off these surfaces and arrives at the receiver at slightly different times, potentially causing distortion and reducing signal clarity.

3. Dynamic Obstacle:
   - Movement of Vehicles: The movement of the ambulance and other nearby vehicles means that the nature of the obstacle is dynamic. As these vehicles move relative to Unit 1, the degree and nature of the signal obstruction and reflection change, causing fluctuating signal quality.

Environmental Factors
- Clear Weather: Beneficial for wireless communication with minimal signal attenuation due to atmospheric conditions.
- Sunlight: Bright sunlight can cause glare, affecting optical sensors and cameras.

Summary of Scene with Focus on Obstacles
The physical environment includes a well-lit highway with multiple vehicles, including a very close ambulance that serves as a significant obstacle. The proximity of the ambulance and other vehicles creates substantial blockage and reflection of signals, leading to potential issues like signal shadowing and multipath interference. The clear weather aids in maintaining good signal propagation conditions, though the dynamic nature of the environment with moving vehicles exacerbates signal fluctuations. The sparse vegetation is unlikely to significantly impact signal strength but can contribute to minor signal attenuation and scattering.
""",

    """
Physical Environment Description

Scene Overview
Location: The environment is an outdoor urban area, likely part of a campus or business district, characterized by a parking lot with adjacent buildings and greenery.
Weather: Clear skies with bright sunlight, indicating a sunny day.

Objects and Distances
Person in Red Shirt:
Position: Standing in the middle of the parking lot.
Distance to Unit 1: Approximately 5 meters, based on the image perspective.
Action: Waving or gesturing towards a nearby car.

Vehicles:
Blue Car: Directly in front of Unit 1, parked. Estimated distance: approximately 8 meters.
White Car: Visible in the parking lot, partially obscured by trees. Estimated distance: around 10-12 meters.
Truck: Positioned further back, approximately 15-17 meters from Unit 1.

Trees and Greenery:
Left Side: One tree is directly next to the parking lot.
Right Side: Several trees are providing shade and contributing to the scenic setting.

Building:
A large building in the background, possibly a hotel or an apartment complex.

LiDAR Object Detection Results
Vehicle Detection:
11.37 meters away at 229.76 degrees: Positioned in the lower left quadrant.
16.97 meters away at 181.48 degrees: Directly behind Unit 1.
9.83 meters away at 241.20 degrees: Also in the lower left quadrant, closer than the previous vehicle.

GPS Module Information
Transmitter (Unit 2):
Distance: 8.08 meters away.
Bearing: 182.45 degrees from Unit 1 (essentially directly south).

Potential Obstacles
Vehicles: Multiple parked vehicles at varying distances can create potential obstacles, causing blockage or reflection of signals.
Trees and Greenery: Trees can obstruct the line of sight (LoS) and attenuate signals, especially those in higher frequency ranges (e.g., 2.4 GHz and above).
Pedestrian (Person in Red Shirt): As a temporary obstacle, they may cause minor fluctuations in signal quality due to blockage or reflection.

Environmental Factors
Clear Weather: Beneficial for wireless communication with minimal signal attenuation due to atmospheric conditions.
Sunlight: While it does not directly impact RF signals, bright sunlight can cause glare affecting optical sensors and cameras.

Summary of Scene
The physical environment includes a well-lit urban parking lot with multiple parked vehicles, scattered trees, and some pedestrian amenities like benches. The vehicles and trees serve as potential obstacles, impacting the line of sight for wireless communication between Unit 1 and Unit 2. The person in the red shirt adds a temporary dynamic element that could briefly obstruct signals. Overall, the clear weather aids in maintaining good signal propagation conditions, though the vehicles and trees can introduce some interference and attenuation effects.
""",

    """
Physical Environment Description

Scene Overview
Location: The environment is an outdoor urban area, likely part of a campus or business district, characterized by a parking lot with adjacent buildings and greenery.
Weather: Clear skies with bright sunlight, indicating a sunny day.

Objects and Distances
Vehicles:
Blue Car: Visible parked on the right side of the road. Estimated distance: approximately 10 meters.
White Car: Directly in front of Unit 1, parked. Estimated distance: approximately 8 meters.
Red Car: Farther back in the parking lot. Estimated distance: approximately 12-15 meters.
Truck: Positioned further back, approximately 15-17 meters from Unit 1.

Trees and Greenery:
Left Side: One tree is directly next to the parking lot.
Right Side: Several trees are providing shade and contributing to the scenic setting.

Building:
A large building in the background, possibly a hotel or an apartment complex.

LiDAR Object Detection Results
Vehicle Detection:
11.42 meters away at 229.72 degrees: Positioned in the lower left quadrant.
9.71 meters away at 242.43 degrees: Also in the lower left quadrant, closer than the previous vehicle.
17.00 meters away at 181.41 degrees: Directly behind Unit 1.

GPS Module Information
Transmitter (Unit 2):
Distance: 8.20 meters away.
Bearing: 182.09 degrees from Unit 1 (essentially directly south).

Potential Obstacles
Vehicles: Multiple parked vehicles at varying distances can create potential obstacles, causing blockage or reflection of signals.
Trees and Greenery: Trees can obstruct the line of sight (LoS) and attenuate signals, especially those in higher frequency ranges (e.g., 2.4 GHz and above).

Environmental Factors
Clear Weather: Beneficial for wireless communication with minimal signal attenuation due to atmospheric conditions.
Sunlight: While it does not directly impact RF signals, bright sunlight can cause glare affecting optical sensors and cameras.

Summary of Scene
The physical environment includes a well-lit urban parking lot with multiple parked vehicles, scattered trees, and some pedestrian amenities like benches. The vehicles and trees serve as potential obstacles, impacting the line of sight for wireless communication between Unit 1 and Unit 2. The clear weather aids in maintaining good signal propagation conditions, though the vehicles and trees can introduce some interference and attenuation effects.
""",

    """
Physical Environment Description

Scene Overview
Location: The environment is an urban city street, characterized by multiple vehicles, buildings, and pedestrians.
Weather: Clear skies with bright sunlight, indicating a sunny day.

Objects and Distances
Vehicles:
White Car: Directly to the left of Unit 1, parked on the side of the road. Estimated distance: approximately 3-5 meters.
Black Car: Positioned farther down the road, on the same side as the white car. Estimated distance: approximately 8-10 meters.
Green Bus: Driving down the street, passing by the parked cars. Estimated distance: approximately 15-20 meters.

Pedestrians:
Left Side: Two people walking on the sidewalk, approximately 10-15 meters away.
Right Side: One person standing on the sidewalk, approximately 5-10 meters away.

Buildings:
Several buildings are visible in the background, providing context for the urban setting.

Traffic Lights and Signs:
Traffic lights are visible at the intersection ahead, about 20-30 meters away.
Street signs provide additional context for the urban environment.

LiDAR Object Detection Results
Vehicle Detection:
14.37 meters away at 338.02 degrees: Likely one of the parked cars or a moving vehicle in the upper left quadrant relative to Unit 1.
16.26 meters away at 181.69 degrees: Directly behind Unit 1.

GPS Module Information
Transmitter (Unit 2):
Distance: 9.52 meters away.
Bearing: 358.70 degrees from Unit 1 (essentially directly north).

Potential Obstacles
Vehicles: Multiple parked and moving vehicles at varying distances can create potential obstacles, causing blockage or reflection of signals.
Pedestrians: Pedestrians can cause temporary signal blockage or reflection, adding dynamic elements to the environment.
Buildings and Structures: Buildings on either side of the street can obstruct the line of sight (LoS) and attenuate signals, especially those in higher frequency ranges.

Environmental Factors
Clear Weather: Beneficial for wireless communication with minimal signal attenuation due to atmospheric conditions.
Sunlight: Bright sunlight can cause glare, affecting optical sensors and cameras.

Summary of Scene
The physical environment includes a well-lit urban city street with multiple vehicles, pedestrians, and buildings. The parked and moving vehicles, as well as the pedestrians, serve as potential obstacles, impacting the line of sight for wireless communication between Unit 1 and Unit 2. The buildings on either side of the street can introduce additional interference and attenuation effects. Overall, the clear weather aids in maintaining good signal propagation conditions, though the dynamic nature of the environment with vehicles and pedestrians can cause fluctuations in signal quality.
""",

    """
Physical Environment Description

Scene Overview
Location: The environment is an urban city street, characterized by multiple vehicles, buildings, and natural elements like trees and shrubs.
Weather: Clear skies with bright sunlight, indicating a sunny day.

Objects and Distances
1. Vehicles:
   - Vehicle 1: Detected 15.51 meters away at a direction of 66.55 degrees.
   - Vehicle 2: Detected 22.92 meters away at a direction of 29.51 degrees.
   - Vehicle 3: Detected 18.97 meters away at a direction of 80.52 degrees.
   - Vehicle 4: Detected 12.68 meters away at a direction of 201.62 degrees.
   - Vehicle 5: Detected 3.70 meters away at a direction of 205.08 degrees.
   - Vehicle 6: Detected 26.97 meters away at a direction of 116.24 degrees.

2. Pedestrians:
   - Second Image: No pedestrians are visible in the immediate vicinity.

3. Buildings:
   - Left Side: Several buildings, including a mix of residential and commercial structures, such as apartment complexes and storefronts.
   - Right Side: Multiple buildings, primarily residential, with some greenery around.

4. Trees and Greenery:
   - Trees and shrubs line the street, providing a natural touch to the urban environment.
   - First Image: Trees are visible on both sides of the street, adding greenery to the scene.

5. Sidewalks and Street Elements:
   - First Image: Sidewalks on both sides of the street, with some shrubs and small plants near the sidewalk.
   - Second Image: Clear sidewalks with visible trees and a fence.

GPS Module Information
- Transmitter (Unit 2):
  - Distance: 134.90 meters away.
  - Bearing: 178.87 degrees from Unit 1.

Potential Obstacles
1. Vehicles:
   - Multiple vehicles at varying distances can create potential obstacles, causing blockage or reflection of signals.
2. Trees and Greenery:
   - Trees can obstruct the line of sight (LoS) and attenuate signals, especially those in higher frequency ranges (e.g., 2.4 GHz and above).
3. Buildings and Structures:
   - Buildings on either side of the street can obstruct the line of sight (LoS) and attenuate signals.

Environmental Factors
- Clear Weather: Beneficial for wireless communication with minimal signal attenuation due to atmospheric conditions.
- Sunlight: Bright sunlight can cause glare, affecting optical sensors and cameras.

Summary of Scene
The physical environment includes a well-lit urban city street with multiple vehicles, buildings, and trees. The vehicles and trees serve as potential obstacles, impacting the line of sight for wireless communication between Unit 1 and Unit 2. The buildings on either side of the street can introduce additional interference and attenuation effects. The clear weather aids in maintaining good signal propagation conditions, though the dynamic nature of the environment with vehicles can cause fluctuations in signal quality.
""",

    """
Physical Environment Description

Scene Overview
- Location: The environment is an urban city street, characterized by multiple vehicles, buildings, and natural elements like trees and shrubs.
- Weather: Clear skies with bright sunlight, indicating a sunny day.

Objects and Distances
1. Vehicles:
   - First Image:
     - Vehicle 1: White car visible on the right lane, approximately 20 meters ahead.
     - Vehicle 2: Other vehicles in the middle distance, about 50-100 meters ahead.
   - Second Image:
     - Vehicle 1: White car on the left lane, approximately 5 meters away.
     - Vehicle 2: Blue car farther down the street, about 30 meters away.
     - Vehicle 3: Black car, approximately 35 meters away.

2. Pedestrians:
   - No pedestrians visible in these images.

3. Buildings:
   - Left Side: Residential and commercial buildings, with some greenery around.
   - Right Side: Primarily commercial buildings with visible greenery.

4. Trees and Greenery:
   - Trees and shrubs line the street, providing a natural touch to the urban environment.
   - Visible greenery and bushes along the sidewalks and in front of buildings.

5. Sidewalks and Street Elements:
   - First Image: Sidewalks on both sides of the street with some shrubs and small plants near the sidewalk.
   - Second Image: Clear sidewalks with visible trees and some residential structures.

LiDAR Object Detection Results
- Vehicle Detection:
  - 10.09 meters away at 345.35 degrees: Vehicle likely in the front view of Unit 1.
  - 18.56 meters away at 319.09 degrees: Vehicle detected further ahead.
  - 10.38 meters away at 204.80 degrees: Vehicle positioned slightly behind Unit 1.

GPS Module Information
- Transmitter (Unit 2):
  - Distance: 16.45 meters away.
  - Bearing: 171.16 degrees from Unit 1 (essentially directly south).

Potential Obstacles
1. Vehicles:
   - Multiple vehicles at varying distances create potential obstacles, causing blockage or reflection of signals.
2. Trees and Greenery:
   - Trees and shrubs can obstruct the line of sight (LoS) and attenuate signals, especially those in higher frequency ranges.
3. Buildings and Structures:
   - Buildings on either side of the street can obstruct the LoS and attenuate signals.

Environmental Factors
- Clear Weather: Beneficial for wireless communication with minimal signal attenuation due to atmospheric conditions.
- Sunlight: Bright sunlight can cause glare, affecting optical sensors and cameras.

Summary of Scene
The physical environment includes a well-lit urban city street with multiple vehicles, buildings, and trees. The vehicles and trees serve as potential obstacles, impacting the line of sight for wireless communication between Unit 1 and Unit 2. The buildings on either side of the street can introduce additional interference and attenuation effects. The clear weather aids in maintaining good signal propagation conditions, though the dynamic nature of the environment with vehicles can cause fluctuations in signal quality.
""",

    """
Physical Environment Description with Focus on Nearby Vehicles as Obstacles

Scene Overview
- Location: The environment is a busy highway, characterized by multiple lanes, vehicles, and urban elements such as buildings and palm trees.
- Weather: Clear skies with bright sunlight, indicating a sunny day.

Objects and Distances
1. Vehicles:
   - First Image:
     - White Car: Visible in the right lane, approximately 15-20 meters from Unit 1.
     - Other Vehicles: Various vehicles are seen ahead at distances ranging from 30 to 100 meters.
   - Second Image:
     - Red Car: Visible in the center lane, approximately 7-10 meters from Unit 1.
     - White Car: In the right lane, approximately 15 meters ahead.
     - Other Vehicles: Various vehicles, including trucks, at distances ranging from 20 to 100 meters.

2. Buildings:
   - IKEA Building: Large blue building visible on the right side of the highway, approximately 50-100 meters away.

3. Trees and Greenery:
   - Palm trees line the highway, adding a scenic element to the urban environment.
   - Sparse desert vegetation visible on the left side of the highway, approximately 20-30 meters from the road.

LiDAR Object Detection Results
- Vehicle Detection:
  - 15.08 meters away at 20.64 degrees: Likely the white car in the right lane.
  - 9.64 meters away at 187.22 degrees: Vehicle detected behind Unit 1.
  - 7.73 meters away at 158.20 degrees: Red car in the center lane.

GPS Module Information
- Transmitter (Unit 2):
  - Distance: 30.48 meters away.
  - Bearing: 5.20 degrees from Unit 1 (essentially directly north).

Impact of Nearby Vehicles as Obstacles
1. Signal Blockage:
   - Close Proximity of Vehicles: The white car approximately 15-20 meters away and the red car 7-10 meters away pose significant obstacles. Their proximity can block the line of sight (LoS) for wireless signals, causing shadowing where signals cannot pass through the metal bodies of the vehicles, resulting in areas of weak or no signal immediately behind them.

2. Signal Reflection and Multipath Interference:
   - Metal Surfaces: The metal surfaces of the nearby cars reflect signals, causing multipath interference. This occurs when the signal bounces off these surfaces and arrives at the receiver at slightly different times, potentially causing distortion and reducing signal clarity.

3. Dynamic Obstacles:
   - Movement of Vehicles: The movement of the cars and other vehicles means that the nature of the obstacle is dynamic. As these vehicles move relative to Unit 1, the degree and nature of the signal obstruction and reflection change, causing fluctuating signal quality.

Environmental Factors
- Clear Weather: Beneficial for wireless communication with minimal signal attenuation due to atmospheric conditions.
- Sunlight: Bright sunlight can cause glare, affecting optical sensors and cameras.

Summary of Scene with Focus on Obstacles
The physical environment includes a well-lit highway with multiple vehicles, including very close cars that serve as significant obstacles. The proximity of the cars creates substantial blockage and reflection of signals, leading to potential issues like signal shadowing and multipath interference. The clear weather aids in maintaining good signal propagation conditions, though the dynamic nature of the environment with moving vehicles exacerbates signal fluctuations. The palm trees and sparse vegetation are unlikely to significantly impact signal strength but can contribute to minor signal attenuation and scattering. The large IKEA building is a notable landmark, providing context for the location.
""",

    """
Physical Environment Description with Focus on Nearby Vehicles as Obstacles

Scene Overview
- Location: The environment is a busy highway, characterized by multiple lanes, vehicles, and urban elements such as buildings and signage.
- Weather: Clear skies with bright sunlight, indicating a sunny day.

Objects and Distances
1. Vehicles:
   - First Image:
     - Red Minivan: Visible in the center lane, approximately 8-10 meters from Unit 1. The proximity of this vehicle makes it a significant obstacle.
     - White Car: In the right lane, approximately 12-15 meters ahead.
     - Blue Car: Slightly further ahead in the center lane, approximately 20-25 meters from Unit 1.
     - Other Vehicles: Various vehicles seen ahead at distances ranging from 30 to 100 meters.
   - Second Image:
     - White Car: Visible on the left lane, approximately 8-10 meters from Unit 1.
     - Silver Car: In the center lane, approximately 20-25 meters ahead.
     - Other Vehicles: Various vehicles, including trucks, at distances ranging from 20 to 100 meters.

2. Buildings and Signage:
   - U-Haul Technical Center: Visible on the right side of the highway, approximately 50-100 meters away.
   - Other Buildings: Industrial and commercial buildings are visible along the highway.

3. Vegetation:
   - Sparse desert vegetation visible along the sides of the highway, approximately 10-20 meters from the road.

LiDAR Object Detection Results
- Vehicle Detection:
  - 8.18 meters away at 11.21 degrees: Likely the red minivan in the center lane.
  - 9.27 meters away at 189.68 degrees: Vehicle detected behind Unit 1.
  - 8.45 meters away at 163.50 degrees: White car in the left lane.

GPS Module Information
- Transmitter (Unit 2):
  - Distance: 30.78 meters away.
  - Bearing: 3.66 degrees from Unit 1 (essentially directly north).

Impact of Nearby Vehicles as Obstacles
1. Signal Blockage:
   - Close Proximity of Vehicles: The red minivan approximately 8-10 meters away and the white car 8-10 meters away pose significant obstacles. Their proximity can block the line of sight (LoS) for wireless signals, causing shadowing where signals cannot pass through the metal bodies of the vehicles, resulting in areas of weak or no signal immediately behind them.

2. Signal Reflection and Multipath Interference:
   - Metal Surfaces: The metal surfaces of the nearby cars reflect signals, causing multipath interference. This occurs when the signal bounces off these surfaces and arrives at the receiver at slightly different times, potentially causing distortion and reducing signal clarity.

3. Dynamic Obstacles:
   - Movement of Vehicles: The movement of the cars and other vehicles means that the nature of the obstacle is dynamic. As these vehicles move relative to Unit 1, the degree and nature of the signal obstruction and reflection change, causing fluctuating signal quality.

Environmental Factors
- Clear Weather: Beneficial for wireless communication with minimal signal attenuation due to atmospheric conditions.
- Sunlight: Bright sunlight can cause glare, affecting optical sensors and cameras.

Summary of Scene with Focus on Obstacles
The physical environment includes a well-lit highway with multiple vehicles, including very close cars that serve as significant obstacles. The proximity of the cars creates substantial blockage and reflection of signals, leading to potential issues like signal shadowing and multipath interference. The clear weather aids in maintaining good signal propagation conditions, though the dynamic nature of the environment with moving vehicles exacerbates signal fluctuations. The buildings and signage, such as the U-Haul Technical Center, provide context for the location, while the sparse vegetation contributes to the overall scene without significantly impacting signal strength.
""",

    """
The given data and images provide a comprehensive view of a busy urban environment, specifically under an overpass and at an intersection. Here is a detailed description of the physical environment, including distances of objects and potential obstacles:

1. General Setting:
   - The images capture a busy urban intersection underneath an overpass. The setting includes multiple lanes of traffic, vehicles, and traffic signals.
   - The scene is well-lit, with clear weather and sunlight visible in the sky.

2. Vehicles and Obstacles:
   - Proximity to Transmitter: The transmitter (Unit 2) is 10.54 meters away from the receiver (Unit 1) at a bearing of 159.90 degrees.
   - Detected Vehicles:
     - A vehicle detected 21.48 meters away at 314.58 degrees.
     - A vehicle detected 24.33 meters away at 336.26 degrees.
     - A vehicle detected 11.60 meters away at 168.20 degrees.
     - A vehicle detected 15.38 meters away at 188.91 degrees.
     - A vehicle detected 23.36 meters away at 163.50 degrees.
   - The vehicles detected by LiDAR are within close range, with the closest vehicle being just over 11 meters away. These vehicles are potential obstacles and must be navigated carefully.

3. Image Analysis:
   - First Image:
     - The image shows a view from under an overpass. There are multiple cars visible, some parked on the side and others moving through the intersection.
     - The presence of a directional sign (curved arrow) suggests lane changes or merges, which could be potential points of congestion or conflict.
     - The underpass creates areas of shadow, which might obscure visibility and create additional hazards.
   - Second Image:
     - This image provides a broader view of the intersection. Multiple vehicles, including a white SUV and various sedans, are navigating the intersection.
     - The traffic flow is regulated by traffic lights, which are visible in the distance.
     - The mixture of moving and stationary vehicles, along with the traffic signals, indicates a complex traffic environment requiring careful navigation.

4. Obstacles and Challenges:
   - Parked Vehicles: Cars parked along the sides of the road under the overpass can limit maneuvering space and create potential blind spots.
   - Moving Traffic: Vehicles moving in different directions, especially near the intersection, increase the likelihood of potential conflicts.
   - Lighting Conditions: The shadow cast by the overpass can create low-visibility areas, posing a challenge for both human drivers and autonomous systems.

5. Additional Details:
   - The environment includes typical urban features like traffic lights, road signs, and diverse vehicle types.
   - The road appears to be well-maintained, but the presence of an overpass suggests a multi-layered traffic system with potential complexity in navigation.

In conclusion, the physical environment depicted involves navigating a busy urban intersection with a mixture of parked and moving vehicles, variable lighting conditions due to the overpass, and multiple potential obstacles identified by the LiDAR system. These factors collectively create a challenging environment for both human and autonomous navigation.
""",

    "The vehicle is detected 9.64 meters away at a direction of 187.22 degrees from Unit 1.",

    "The car in the center lane is approximately 7-10 meters away from Unit 1.",

    "The nearest tree is directly next to the parking lot Unit 1.",

    "The bus is approximately 15-20 meters away from Unit 1."
    ]

In [ ]:
def get_result(conversational_chain, model_name):
    results = []
    contexts = []
    for question in questions:
        result = conversational_chain.invoke({"query": question})
        results.append(result['result'])
        sources = result['source_documents']

        contents = []
        for i in range(len(sources)):
            contents.append(sources[i].page_content)
        contexts.append(contents)

    d = {
        "question": questions,
        "answer": results,
        "contexts": contexts,
        "ground_truth": ground_truths
        }

    dataset = Dataset.from_dict(d)
    score = evaluate(dataset,
                     embeddings=embeddings,
                     metrics=[faithfulness, answer_relevancy, context_precision, context_recall, context_entity_recall,
                              answer_similarity, answer_correctness])
    score_df = score.to_pandas()
    score_df.to_csv("EvaluationScores_" + model_name + ".csv", encoding="utf-8", index=False)

    return score_df

In [ ]:
scores_mistral = get_result(conversational_chain=conversational_chain_mistral_7b, model_name="mistral_7b")

In [ ]:
scores_mistral[['faithfulness', 'answer_relevancy', 'context_precision', 'context_recall',
                'context_entity_recall', 'answer_similarity', 'answer_correctness']].mean(axis=0)

In [ ]:
scores_mixtral = get_result(conversational_chain_mixtral_8x7b, "mixtral_8x7b")

In [ ]:
scores_mixtral[['faithfulness', 'answer_relevancy', 'context_precision', 'context_recall',
                'context_entity_recall', 'answer_similarity', 'answer_correctness']].mean(axis=0)

In [ ]:
scores_llama_8b = get_result(conversational_chain_llama_8b, "llama_8b")

In [ ]:
scores_llama_8b[['faithfulness', 'answer_relevancy', 'context_precision', 'context_recall',
                 'context_entity_recall', 'answer_similarity', 'answer_correctness']].mean(axis=0)

In [ ]:
scores_llama_70b = get_result(conversational_chain_llama_70b, "llama_70b")

In [ ]:
scores_llama_70b[['faithfulness', 'answer_relevancy', 'context_precision', 'context_recall',
                  'context_entity_recall', 'answer_similarity', 'answer_correctness']].mean(axis=0)

In [ ]:
scores_llama_405b = get_result(conversational_chain_llama_405b, "llama_405b")

In [ ]:
scores_llama_405b[['faithfulness', 'answer_relevancy', 'context_precision', 'context_recall',
                   'context_entity_recall', 'answer_similarity', 'answer_correctness']].mean(axis=0)

In [ ]:
question_sample = ["""Given the following information captured from unit1, describe the physical environment this information provides and be as detailed as possible. Do your best to estimate distances of objects and their types, if there are any, and if there are any blockages from obstacles, cars, pedestrians or anything that can cause obstacles between unit 1 and unit 2 and how line-of-sight might be affected between the two. Unit1 is a vehicle with a 360 camera (two 180 degree cameras), 360 degree 3D lidar, and a GPS module, unit2 is a vehicle mounted with a GPS module and a transmitter. 

GPS: 
The transmitter (Unit 2) is 7.7534236928011895 meters away at a bearing of 358.8162531212902 degrees from the receiver (Unit 1).

LiDAR object detection results:
A Vehicle is detected 16.77 meters away at a direction of 350.83 degrees. 
A Vehicle is detected 16.08 meters away at a direction of 335.32 degrees. 
A Vehicle is detected 13.74 meters away at a direction of 280.51 degrees. 
A Vehicle is detected 16.64 meters away at a direction of 195.72 degrees. 
A Vehicle is detected 8.07 meters away at a direction of 182.27 degrees.
A Vehicle is detected 15.04 meters away at a direction of 161.45 degrees. 
A Pedestrian is detected 17.08 meters away at a direction of 35.32 degrees.
A Cyclist is detected 16.64 meters away at a direction of 55.72 degrees.

Scene details from camera: 
The scene depicts a busy city street filled with cars, trucks, and motorcycles. There are several vehicles on the road, including a silver car, a white car, a truck, and a motorcycle. Some of the vehicles are parked on the side of the street, while others are moving along the road. The sun is shining brightly in the sky, casting a warm and inviting glow over the entire scene.The scene is captured from the back of a car driving down a busy city street. There are several cars in front of the car, creating a congested traffic situation. Some of these cars are parked on the side of the street, while others are moving along with the flow of traffic. In addition to the cars, there are several trees and palm trees lining the street, adding to the scenic view.""", "."]

In [ ]:
ground_truth_sample =["""Based on the information provided from Unit 1's sensors (GPS, LiDAR, and cameras), here’s a detailed description of the physical environment between Unit 1 and Unit 2, as well as the broader surroundings:

1. **Location and Relative Positioning**
   - Unit 2 (Vehicle with GPS and transmitter) is located 7.75 meters away from Unit 1, almost directly north, at a bearing of 358.82 degrees. This suggests that Unit 2 is quite close and positioned slightly to the right of the true north direction.
   - The space between Unit 1 and Unit 2 is relatively clear, with no major obstacles directly blocking the path.
   -  No specific obstacles like pedestrians, bicycles, or stationary objects were mentioned within the immediate vicinity are affecting the interaction between Unit 1 and Unit 2 and ensures direct line-of-sight communication between the two vehicles.
   - Between Unit 1 and Unit 2, however, it seems relatively clear since Unit 2 is only about 7.75 meters away, indicating minimal obstructions along this line-of-sight path

2. **LiDAR Object Detection** 
LiDAR detects multiple objects, primarily vehicles, around Unit 1, providing insight into the surrounding traffic and congestion:
   - Vehicle 1: Detected 16.77 meters away at a bearing of 350.83 degrees, slightly north-northwest of Unit 1. This vehicle might be moving in traffic or parked on the side of the road but is distant enough not to block the immediate path.
   - Vehicle 2: Detected 16.08 meters away at a bearing of 335.32 degrees, positioned northwest of Unit 1. This vehicle, too, is somewhat far, possibly contributing to the congestion.
   - Vehicle 3: Detected 13.74 meters away at a bearing of 280.51 degrees, which places it west of Unit 1. This suggests that this vehicle could be parked or moving on the other side of the road or in a nearby lane.
   - Vehicle 4: Detected 16.64 meters away at a bearing of 195.72 degrees, south-southwest of Unit 1. It’s distant enough not to be an immediate concern for obstruction.
   - Vehicle 5: Detected 8.07 meters away at a bearing of 182.27 degrees, almost directly south of Unit 1. This vehicle is relatively close, which may indicate another car in close proximity to Unit 1, possibly contributing to the traffic.
   - Vehicle 6: Detected 15.04 meters away at a bearing of 161.45 degrees, southeast of Unit 1. Like other detected vehicles, this could be part of the ongoing congestion or parked.
   - Pedestrian: Detected 17.08 meters away at a bearing of 35.32 degrees, located northeast of Unit 1. This person is farther away but could be moving on the sidewalk or crossing the street.
   - Cyclist: Detected 16.64 meters away at a bearing of 55.72 degrees, located east-northeast of Unit 1. The cyclist may be moving parallel to traffic or on the side of the road, relatively far from Unit 1.

3. **Camera Description of the Scene**
The camera provides a visual context, showing a busy city street filled with vehicles, including cars, trucks, motorcycles, and trees and palm trees lining the street. The scene is one of moderate to heavy traffic, with several vehicles parked along the side of the road while others move forward, contributing to a congested traffic situation. The bright sunlight suggests that visibility is good, but the sun may cause some glare or shadows.

4. **Obstacles and Potential Blockages**
   - The detected vehicles and pedestrians form a somewhat congested environment, though none of the vehicles are directly between Unit 1 and Unit 2 based on their relative bearings.
   - However, with multiple vehicles surrounding Unit 1, especially one 8.07 meters away to the south (Vehicle 5), this vehicle is close enough to potentially obstruct the path depending on traffic conditions.
   - The pedestrian and cyclist, although not in the direct path between Unit 1 and Unit 2, suggest additional dynamic elements in the scene. The pedestrian is more distant, but the cyclist, being closer, may need to be watched for movement, as cyclists are typically faster than pedestrians and could be harder to track as they move through traffic.
   - Parked cars along the side of the street are contributing to the congestion but don’t directly block the immediate space between Unit 1 and Unit 2.

5. **Estimated Distances and Spatial Relationships**
   - From the detected bearings and distances, the environment around Unit 1 can be described as moderately busy, with vehicles scattered mostly between 8 and 17 meters away, indicating nearby traffic but no immediate blockages.
   - The closest objects detected are Unit 2 (7.75 meters away) and Vehicle 5 (8.07 meters away). However, Unit 2 is almost directly north, while Vehicle 5 is to the south, so they do not interfere with each other.

Conclusion:
Unit 1 is navigating a congested urban environment with multiple vehicles and a cyclist nearby, but the path to Unit 2 is relatively clear, as no major obstacles directly block it. Unit 1 is surrounded by moving and parked vehicles, suggesting a slow-moving traffic flow, but none of the detected vehicles or objects create an immediate obstruction between the two units. The scene includes pedestrians, cyclists, and various vehicles, contributing to a dynamic environment that may require careful navigation and monitoring, especially as traffic and other elements move in the scene.""",""] 


In [ ]:
llm_mixtral_8x7b_sample = get_llm("mistralai/Mixtral-8x7B-Instruct-v0.1")
conversational_chain_mixtral_8x7b_sample = get_conversation_chain_mistral(llm_mixtral_8x7b_sample, vector_store)

In [ ]:
print(conversational_chain_mixtral_8x7b_sample.invoke({"query": question_sample}))

In [ ]:
results = []
contexts = []

for question in question_sample:
    result = conversational_chain_mixtral_8x7b_sample.invoke({"query": question})
    results.append(result['result'])
    sources = result['source_documents']
    
    contents = []
    for i in range(len(sources)):
        contents.append(sources[i].page_content)
    contexts.append(contents)

d = {
    "question": question_sample,
    "answer": results,
    "contexts": contexts,
    "ground_truth": ground_truth_sample
    }

dataset = Dataset.from_dict(d)
print(dataset)
score = evaluate(dataset,
                 embeddings=embeddings,
                 metrics=[faithfulness, answer_relevancy, context_recall,
                          answer_similarity, answer_correctness])
score_df_sample_chain = score.to_pandas()

In [ ]:
score_df_sample_chain

In [ ]:
results = []

for question in question_sample:
    result = llm_mixtral_8x7b_sample.invoke(question)
    results.append(result)


d = {
    "question": question_sample,
    "answer": results,
    "contexts": contexts,
    "ground_truth": ground_truth_sample
    }

dataset = Dataset.from_dict(d)
score = evaluate(dataset,
                 embeddings=embeddings,
                 metrics=[faithfulness, answer_relevancy, context_precision,
                          answer_similarity, answer_correctness])
score_df_sample_vanilla = score.to_pandas()

In [ ]:
score_df_sample_vanilla